## Pré-processamento

Segundo a importância das features, grande parte das features construídas apresentam importância significativa, logo, serão mantidas. Além disso, faremos o pré-processamento do texto, removendo stopwords, convertendo as palavras para seu lema, convertendo para minúsculas, a fim de simplificar o modelo, contendo apenas informação de quantas palavras em caixa alta estavam presente.

Dentre os caracteres especiais, iremos remover todos, exceto interrogações e exclamações, pois apesar de não serem estritamente relevantes a maioria dos textos que tem sentimento positivo ou negativo, possuem uma maior quantidade desses caracteres. Assim, podemos considerar que eles podem carregar alguma informação relevante.

Dessa forma, precisaremos extrair informações da sentença de forma mais granular, ou seja, palavra por palavra. Para isso, utilizaremos o TF-IDF (Term Frequency-Inverse Document Frequency), que é uma técnica de transformação de texto em vetores numéricos, onde cada palavra é representada por um número que indica sua importância no texto.

In [26]:
df_processed, test_results = run_sentiment_analysis(
    df=dataset_df,  # Seu DataFrame original
    lm_file_path=LM_FILE_PATH,
    use_balanced=False,  # Mude para False se não quiser balancear
    print_plot=False  # Desativar plotagens para pré-processamento
)

Dicionário LM carregado: ../data/Loughran-McDonald_MasterDictionary_1993-2024.csv
  Positive: 347 palavras
  Negative: 2345 palavras
  Uncertainty: 297 palavras
  Litigious: 903 palavras
  Strong_Modal: 19 palavras
  Weak_Modal: 27 palavras
  Constraining: 184 palavras

Criando features...

Usando dataset original: 5842 amostras

Distribuição das classes:
Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

Testando significância estatística...
Testando 12 features (alpha=0.05)

Feature: text_length
  Kruskal-Wallis: H=98.386, p=0.0000 - Significativo

Feature: word_count
  Kruskal-Wallis: H=38.185, p=0.0000 - Significativo

Feature: avg_word_length
  Kruskal-Wallis: H=38.859, p=0.0000 - Significativo

Feature: exclamation_count
  Kruskal-Wallis: H=44.736, p=0.0000 - Significativo

Feature: question_count
  Kruskal-Wallis: H=25.719, p=0.0000 - Significativo

Feature: uppercase_ratio
  Kruskal-Wallis: H=348.114, p=0.0000 - Significativo

Feature: lm_po

In [27]:
# Remover alfanuméricos
dataset_df['Sentence'] = dataset_df['Sentence'].str.replace(r'\d+', '')

# Deixar tudo em minúsculo
dataset_df['Sentence'] = dataset_df['Sentence'].str.lower()

# Remover pontuação, exceto exclamações e interrogações.
dataset_df['Sentence'] = dataset_df['Sentence'].str.replace(r'[^\w\s!?]', '', regex=True)


# Remover stopwords (Ex: "the", "is", "in", "and")
stopwords = nltk.corpus.stopwords.words('english')
dataset_df['Sentence'] = dataset_df['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

# Leamatização (reduzir palavras à sua raiz)
lemmatizer = WordNetLemmatizer()
dataset_df['Sentence'] = dataset_df['Sentence'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

In [28]:
# 1. Primeiro fazer o train_test_split nos índices
train_idx, test_idx = train_test_split(
    dataset_df.index,
    test_size=0.2,
    random_state=42
)

# 2. Separar os dados de treino e teste usando os índices
X_train_text = dataset_df.loc[train_idx, 'Sentence']
X_test_text = dataset_df.loc[test_idx, 'Sentence']


X_train_num = dataset_df.loc[train_idx, numerical_features_names]
X_test_num = dataset_df.loc[test_idx, numerical_features_names]

# 3. Aplicar TF-IDF apenas no texto
tfidf_vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

# 4. Combinar TF-IDF com features numéricas
X_train_combined = np.hstack((X_train_tfidf.toarray(), X_train_num))

# 4.1 Normalizar as features

scaler = StandardScaler()
X_train_combined = scaler.fit_transform(X_train_combined)

X_test_combined = np.hstack((X_test_tfidf.toarray(), X_test_num))
X_test_combined = scaler.transform(X_test_combined)

y_train = dataset_df['Sentiment'].loc[train_idx]
y_test = dataset_df['Sentiment'].loc[test_idx]


# 6. Aplicar Label Encoder nos targets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)